# Explore here

In [9]:
import pandas as pd
import numpy as np
import regex as re

import matplotlib.pyplot as plt
import seaborn as sns

from nltk import download
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report

from sklearn.svm import SVC

import warnings

In [4]:
#eliminar los avisos mientras entrenamos modelos
warnings.filterwarnings("ignore")

In [5]:
df_url = pd.read_csv ("/Users/luiseduardogarciablanco/Desktop/bootcamp/NLP_project/url_spam.csv")
df_url

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,True
1,https://www.hvper.com/,True
2,https://briefingday.com/m/v4n3i4f3,True
3,https://briefingday.com/n/20200618/m#commentform,False
4,https://briefingday.com/fan,True
...,...,...
2994,https://www.smartcitiesworld.net/news/news/dee...,False
2995,https://www.youtube.com/watch,True
2996,https://techcrunch.com/2019/07/04/an-optimisti...,False
2997,https://www.technologyreview.com/2019/12/20/13...,False


In [6]:
df_url.info ()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      2999 non-null   object
 1   is_spam  2999 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 26.5+ KB


In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Descargar recursos necesarios de nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/luiseduardogarciablanco/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/luiseduardogarciablanco/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/luiseduardogarciablanco/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
df_url = df_url.drop_duplicates()
df_url = df_url.reset_index(inplace = False, drop = True)
df_url.shape

(2369, 2)

In [12]:
# Inicializar el lematizador
lemmatizer = WordNetLemmatizer()

# Función para preprocesar URLs
def preprocess_url(url):
    tokens = word_tokenize(url)
    tokens = [token.lower() for token in tokens if token.isalnum()]  # Filtramos solo tokens alfanuméricos
    stop_words = set(stopwords.words('english'))  # Usamos stopwords en inglés por simplicidad
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]  # Lematización
    return ' '.join(tokens)

In [13]:
# Preprocesar las URLs
df_url['processed_url'] = df_url['url'].apply(preprocess_url)

In [14]:
# Inicializar el lematizador
lemmatizer = WordNetLemmatizer()

# Función para preprocesar URLs
def preprocess_url(url):
    tokens = word_tokenize(url)
    tokens = [token.lower() for token in tokens if token.isalnum()]  # Filtramos solo tokens alfanuméricos
    stop_words = set(stopwords.words('english'))  # Usamos stopwords en inglés por simplicidad
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]  # Lematización
    return ' '.join(tokens)

In [15]:
# Separar las características y el objetivo
X = df_url['processed_url']
y = df_url['is_spam']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
# Vectorizar las URLs
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [17]:
# Crear y entrenar el modelo SVC
model = SVC(kernel='linear')  # Usamos kernel lineal para simplificar
model.fit(X_train_vec, y_train)


SVC(kernel='linear')

In [18]:
# Hacer predicciones
y_pred = model.predict(X_test_vec)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print(classification_report(y_test, y_pred))

Accuracy: 0.9177215189873418
              precision    recall  f1-score   support

       False       0.92      1.00      0.96       435
        True       0.00      0.00      0.00        39

    accuracy                           0.92       474
   macro avg       0.46      0.50      0.48       474
weighted avg       0.84      0.92      0.88       474

